# Get/Update State

When running LangGraph agents, you can easily get or update the state of the agent at any point in time. This allows for several things. Firstly, it allows you to inspect the state and take actions accordingly. Second, it allows you to modify the state - this can be useful for changing or correcting potential actions.

**Note:** this requires passing in a checkpointer.

## Setup

First we need to install the packages required

In [1]:
!pip install --quiet -U langchain langchain_openai tavily-python


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


Next, we need to set API keys for OpenAI (the LLM we will use) and Tavily (the search tool we will use)

In [2]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API Key:")

OpenAI API Key: ········
Tavily API Key: ········


Optionally, we can set API key for [LangSmith tracing](https://smith.langchain.com/), which will give us best-in-class observability.

In [ ]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangSmith API Key:")

## Set up the tools

We will first define the tools we want to use.
For this simple example, we will use a built-in search tool via Tavily.
However, it is really easy to create your own tools - see documentation [here](https://python.langchain.com/docs/modules/agents/tools/custom_tools) on how to do that.


In [1]:
from langchain_community.tools.tavily_search import TavilySearchResults

tools = [TavilySearchResults(max_results=1)]

We can now wrap these tools in a simple ToolExecutor.
This is a real simple class that takes in a ToolInvocation and calls that tool, returning the output.
A ToolInvocation is any class with `tool` and `tool_input` attribute.


In [2]:
from langgraph.prebuilt import ToolExecutor

tool_executor = ToolExecutor(tools)

## Set up the model

Now we need to load the chat model we want to use.
Importantly, this should satisfy two criteria:

1. It should work with messages. We will represent all agent state in the form of messages, so it needs to be able to work well with them.
2. It should work with OpenAI function calling. This means it should either be an OpenAI model or a model that exposes a similar interface.

Note: these model requirements are not requirements for using LangGraph - they are just requirements for this one example.

In [3]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0)


After we've done this, we should make sure the model knows that it has these tools available to call.
We can do this by converting the LangChain tools into the format for OpenAI function calling, and then bind them to the model class.


In [4]:
from langchain_core.utils.function_calling import convert_to_openai_function

functions = [convert_to_openai_function(t) for t in tools]
model = model.bind_functions(functions)

## Define the nodes

We now need to define a few different nodes in our graph.
In `langgraph`, a node can be either a function or a [runnable](https://python.langchain.com/docs/expression_language/).
There are two main nodes we need for this:

1. The agent: responsible for deciding what (if any) actions to take.
2. A function to invoke tools: if the agent decides to take an action, this node will then execute that action.

We will also need to define some edges.
Some of these edges may be conditional.
The reason they are conditional is that based on the output of a node, one of several paths may be taken.
The path that is taken is not known until that node is run (the LLM decides).

1. Conditional Edge: after the agent is called, we should either:
   a. If the agent said to take an action, then the function to invoke tools should be called
   b. If the agent said that it was finished, then it should finish
2. Normal Edge: after the tools are invoked, it should always go back to the agent to decide what to do next

Let's define the nodes, as well as a function to decide how what conditional edge to take.

In [5]:
from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import FunctionMessage


# Define the function that determines whether to continue or not
def should_continue(messages):
    last_message = messages[-1]
    # If there is no function call, then we finish
    if "function_call" not in last_message.additional_kwargs:
        return "end"
    # Otherwise if there is, we continue
    else:
        return "continue"


# Define the function to execute tools
def call_tool(messages):
    # Based on the continue condition
    # we know the last message involves a function call
    last_message = messages[-1]
    # We construct an ToolInvocation from the function_call
    action = ToolInvocation(
        tool=last_message.additional_kwargs["function_call"]["name"],
        tool_input=json.loads(
            last_message.additional_kwargs["function_call"]["arguments"]
        ),
    )
    # We call the tool_executor and get back a response
    response = tool_executor.invoke(action)
    # We use the response to create a FunctionMessage
    function_message = FunctionMessage(content=str(response), name=action.tool)
    # We return a list, because this will get added to the existing list
    return function_message

## Define the graph

We can now put it all together and define the graph!

In [6]:
from langgraph.graph import MessageGraph, END

# Define a new graph
workflow = MessageGraph()

# Define the two nodes we will cycle between
workflow.add_node("agent", model)
workflow.add_node("action", call_tool)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Finally we pass in a mapping.
    # The keys are strings, and the values are other nodes.
    # END is a special node marking that the graph should finish.
    # What will happen is we will call `should_continue`, and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node will then be called.
    {
        # If `tools`, then we call the tool node.
        "continue": "action",
        # Otherwise we finish.
        "end": END,
    },
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge("action", "agent")

**Persistence**

To add in persistence, we pass in a checkpoint when compiling the graph

In [7]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

In [8]:
# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile(checkpointer=memory, interrupt_before=["action"])

## Interacting with the Agent

We can now interact with the agent. Between interactions you can get and update state.


In [9]:
from langchain_core.messages import HumanMessage

thread = {"configurable": {"thread_id": '3'}}
inputs = HumanMessage(content="hi! I'm bob")
for event in app.stream(inputs, thread):
    for v in event.values():
        print(v)

content='Hello Bob! How can I assist you today?' id='87e8b88a-b28e-4517-a62f-7b9baebd3329'


Let's now get the current state

In [10]:
app.get_state(thread)

StateSnapshot(values=[HumanMessage(content="hi! I'm bob", id='5a84a602-37d9-4903-9c3c-d0f892238580'), AIMessage(content='Hello Bob! How can I assist you today?', id='87e8b88a-b28e-4517-a62f-7b9baebd3329')], next=(), config={'configurable': {'thread_id': '3', 'thread_ts': '2024-04-02T17:13:46.250299+00:00'}}, parent_config=None)

Let's now show how we can update the state. This is very useful when you want to have a human-in-the-loop to validate (and potentially change) the action to take. If you notice above, we added `interrupt_before=["action"]` - this means that before any actions are taken we pause. This is a great moment to allow the user to correct and update the state!

In [11]:
inputs = HumanMessage(content="what is the weather in sf currently")
for event in app.stream(inputs, thread):
    for v in event.values():
        print(v)

content='' additional_kwargs={'function_call': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}} id='8e6ae8d5-a258-47aa-9e2c-1ca3e78add90'


This is the function call the model produced

In [12]:
current_values = app.get_state(thread)
current_values.values[-1].additional_kwargs

{'function_call': {'arguments': '{"query":"current weather in San Francisco"}',
  'name': 'tavily_search_results_json'}}

Let's update the search string before proceeding

In [13]:
current_values.values[-1].additional_kwargs = {'function_call': {'arguments': '{"query":"weather in San Francisco today"}',
  'name': 'tavily_search_results_json'}}

In [14]:
app.update_state(thread, current_values.values)

{'configurable': {'thread_id': '3',
  'thread_ts': '2024-04-02T16:58:19.692203+00:00'}}

The current state now reflects our updated search query!

In [15]:
app.get_state(thread)

StateSnapshot(values=[HumanMessage(content="hi! I'm bob", id='1493cdf4-b7b5-46c9-a3e7-ad3b661fcb92'), AIMessage(content='Hello Bob! How can I assist you today?', id='028e0185-2ca4-4c1d-8660-93c17b839275'), HumanMessage(content='what is the weather in sf currently', id='76d6a5ff-d400-4edb-a264-b584107e2231'), AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"weather in San Francisco today"}', 'name': 'tavily_search_results_json'}}, id='b143d5f6-5216-4a48-b76a-68b2f3eb6ff2')], next=('action',), config={'configurable': {'thread_id': '3', 'thread_ts': '2024-04-02T16:58:19.692203+00:00'}}, parent_config=None)

If we start the agent again it will pick up from the state we updated.

In [16]:
for event in app.stream(None, thread):
    for v in event.values():
        print(v)

content="[{'url': 'https://forecast.weather.gov/zipcity.php?inputstring=San francisco,CA', 'content': 'Detailed Forecast. Today. Mostly sunny, with a high near 62. Light and variable wind becoming west southwest 5 to 8 mph in the afternoon. Tonight. Mostly clear, with a low around 49. West wind 5 to 8 mph becoming north northwest after midnight. Monday. Sunny, with a high near 67.'}]" name='tavily_search_results_json' id='9b33eb74-b079-4615-9c8a-9f41b4e176ba'
content='The weather in San Francisco today is mostly sunny with a high near 62 degrees Fahrenheit. The wind is light and variable, becoming west-southwest at 5 to 8 mph in the afternoon. Tonight, it will be mostly clear with a low around 49 degrees Fahrenheit. Tomorrow is expected to be sunny with a high near 67 degrees Fahrenheit.' id='18ff870d-5d5d-40d9-9eca-72e345749b51'


## Checking history

Let's browse the history of this thread, from newest to oldest.

In [17]:
for state in app.get_state_history(thread):
    print(state)
    print('--')
    if len(state.values) == 4:
        to_replay = state

StateSnapshot(values=[HumanMessage(content="hi! I'm bob", id='1493cdf4-b7b5-46c9-a3e7-ad3b661fcb92'), AIMessage(content='Hello Bob! How can I assist you today?', id='028e0185-2ca4-4c1d-8660-93c17b839275'), HumanMessage(content='what is the weather in sf currently', id='76d6a5ff-d400-4edb-a264-b584107e2231'), AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"weather in San Francisco today"}', 'name': 'tavily_search_results_json'}}, id='b143d5f6-5216-4a48-b76a-68b2f3eb6ff2'), FunctionMessage(content="[{'url': 'https://forecast.weather.gov/zipcity.php?inputstring=San francisco,CA', 'content': 'Detailed Forecast. Today. Mostly sunny, with a high near 62. Light and variable wind becoming west southwest 5 to 8 mph in the afternoon. Tonight. Mostly clear, with a low around 49. West wind 5 to 8 mph becoming north northwest after midnight. Monday. Sunny, with a high near 67.'}]", name='tavily_search_results_json', id='9b33eb74-b079-4615-9c8a-9f41b4e176ba'), AIMe

We can go back to any of these states and restart the agent from there!

In [18]:
to_replay

StateSnapshot(values=[HumanMessage(content="hi! I'm bob", id='1493cdf4-b7b5-46c9-a3e7-ad3b661fcb92'), AIMessage(content='Hello Bob! How can I assist you today?', id='028e0185-2ca4-4c1d-8660-93c17b839275'), HumanMessage(content='what is the weather in sf currently', id='76d6a5ff-d400-4edb-a264-b584107e2231'), AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}}, id='b143d5f6-5216-4a48-b76a-68b2f3eb6ff2')], next=('action',), config={'configurable': {'thread_id': '3', 'thread_ts': '2024-04-02T16:58:18.058645+00:00'}}, parent_config=None)

To replay from this place we just need to pass its config back to the agent.

In [19]:
for event in app.stream(None, to_replay.config):
    for v in event.values():
        print(v)

content="[{'url': 'https://www.accuweather.com/en/us/san-francisco/94103/current-weather/347629', 'content': 'Get the latest weather conditions and forecast for San Francisco, CA. See the temperature, humidity, wind, pressure, cloud cover, and alerts for the current hour and the next few days.'}]" name='tavily_search_results_json' id='73ab98d6-d7cf-4812-9572-f58698394c9f'
content='You can check the current weather conditions and forecast for San Francisco, CA on [AccuWeather](https://www.accuweather.com/en/us/san-francisco/94103/current-weather/347629). This will provide you with information on temperature, humidity, wind, pressure, cloud cover, and alerts for the current hour and the next few days.' id='a54c0531-1683-4d61-87b6-ee7f66971c25'
